# Davies Meyer
Debemos retornar una función de hash que sea una función de compresión a partir de la contrucción de Davies-Meyer.

Por ende, debemos obtener una llave y valor a partir del input, encryptar el valor con la llave, y posteriormente hacer un XOR entre la llave y el valor encriptado. Algo a notar, es que la construcción no requiere necesariamente que la llave y el mensaje sean del mismo tamaño, pero para el XOR se necesita tener que ambos sean del mismo tamaño. Por ende, se trunca al valor más pequeño entre ambos (en python eso se hace por default al usar `zip`).

In [1]:
def davies_meyer(encrypt, l_key: int, l_message: int):
    def hash(key_message):
        key = bytes(key_message[:l_key]) # obtenemos la llave
        message = bytes(key_message[l_key:l_key + l_message]) # obtenemos el valor
        
        encrypted_message = encrypt(key, message) # encriptamos
        
        hashed_message = bytearray(a ^ b for (a, b) in zip(encrypted_message, message)) # hacemos el XOR
       
        return hashed_message
    return hash

# Padding
Debemos aplicar padding. Como visto en clases, se debe rellenar el último bloque con un 1 seguido de los suficientes 0 como para llegar a terminar el último bloque del mismo tamaño del resto. Posteriormente, se agrega un bloque final que indica el tamaño del mensaje original (módulo 2^(largo del bloque)).

In [6]:
import math

def pad(message: bytearray, l_block: int):
    message_length = len(message)
    n_blocks = math.ceil(message_length / l_block) # obtenemos cantidad de bloques originales
    missing_bytes = n_blocks * l_block - message_length # obtenemos cuántos bytes faltan agregar
    
    added_bytes = bytearray()
    if missing_bytes:
        added_bytes += bytearray(b'\x01') # primero un 1
        added_bytes += bytearray(b'\x00'*(missing_bytes - 1)) # rellenamos con 0
    
    coded_length = message_length % (2 ** l_block) # de acuerdo a las diapositivas de clases
    last_block = bytearray(coded_length.to_bytes(l_block,'big'))
    padded_message = message + added_bytes + last_block

    return padded_message
    

# Merke Damgard
A partir de un vector de inicialización, dividimos el mensaje (con padding) en bloques, vamos bloque por bloque generando h_i. Se retorna el h final.

In [3]:
def merkle_damgard(IV: bytearray, comp, l_block: int):
    def hash(message):
        padded_message = pad(message, l_block)
        h = IV # inicialmente empieza el vector de inicialización
        for i in range(0, len(padded_message), l_block):
            block = padded_message[i: i + l_block] # obtenemos el siguiente bloque
            h = comp(block + h) # calculamos el siguiente h
        return h # retornamos el h final
    return hash


In [4]:
#from Crypto.Cipher import AES

#def AES_128(key: bytearray, message: bytearray) -> bytearray:
#    a = AES.new(key, AES.MODE_ECB)
#    return bytearray(a.encrypt(message))

In [5]:
#if __name__ == "__main__":
#    compresion = davies_meyer(AES_128, 16, 16)
#    hash = merkle_damgard(bytearray(b'1234567890123456'), compresion, 16)

#    s1 = bytearray(b'Este es un mensaje de prueba para la tarea 2')
#    s2 = bytearray(b'Este es un mensaje de Prueba para la tarea 2')
#    s3 = bytearray(b'Un mensaje corto')
#    s4 = bytearray(b'')

#    h1 = hash(s1)
#    h2 = hash(s2)
#    h3 = hash(s3)
#    h4 = hash(s4)

#    print(h1)
#    print(h2)
#    print(h3)
#    print(h4)
